In [10]:
import ephem #library untuk menghitung posisi benda langit(deklinasi, ra, dll) berdasarkan lokasi dan waktu tertentu.
import math as m #menyediakan fungsi matematika, ex:sin,cos,tan,radians,degrees,pi,dll
from math import pi #digunakan untuk konversi dari radian ke derajat.
from datetime import datetime, timedelta, timezone #untuk manipulasi tanggal dan waktu
from time import strftime #memformat waktu ke dalam tipe data string
import pandas as pd #Membuat array Menyimpan jadwal sholat dalam format tabel 
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple

try:
    from skyfield.api import load, wgs84  # type: ignore
    from skyfield.api import Star
    _SKYFIELD_AVAILABLE = True
except ImportError:
    # Skyfield is not available in this environment.  The functions will raise
    # informative errors if called without Skyfield installed.
    _SKYFIELD_AVAILABLE = False

### FUNGSI MENGUBAH FORMAT DMS KE DERAJAT DESIMAL

In [11]:
#Fungsi konversi derajat ke desimal
def deg_to_des(deg, menit, detik):
    sign    = -1 if deg < 0 else 1
    desimal = abs(deg)+(menit / 60) + (detik/3600)
    return desimal * sign
#Fungsi konversi desimal ke derajat
def format_dms(desimal):
    # Tentukan tanda positif/negatif untuk milai derajat
    sign = -1 if desimal < 0 else 1
    decimal_degrees = abs(desimal)
    # Hitung derajat
    degrees = int(decimal_degrees) * sign #mengambil nilai bil. bulan untuk derajat dari desimal_degrees Ex: 7.3347 = 7 derajat
    # Hitung menit
    sisa_DD = decimal_degrees - abs(degrees) #mengambil nilai desimal dari decimal_degrees Ex: 7.3347 = 0.3347
    total_minutes = sisa_DD * 60 #mengambil nilai menit desimal dari decimal_degrees Ex: 0.3347 = 20.082 menit
    minutes = int(total_minutes) #mengambil nilai menit DMS dari total_minutes Ex: 20.082 = 20 menit
    # Hitung detik dan tentukan pembulatan
    sisa_menit = (total_minutes - minutes) * 60 #pecahan menit 0.5 setara dengan 30 detik, maka disini pecahan menit * 60 Ex: 20.082 = 0.082 = 4.92 detik
    if sisa_menit >= 30:
        minutes += 1
    # Jika menit mencapai 60, tambahkan ke derajat
    if minutes == 60:
        degrees += 1 if degrees >= 0 else -1
        minutes = 0
    return f"{degrees}° {minutes}'"


### INPUT DATA

In [12]:
#data lintang tempat
def Lintang_tempat(der_lt,mnt_lt,dtk_lt):
    lt_tempat  = {"derajat": der_lt, "menit" : mnt_lt, "detik" : dtk_lt} #satuan derajat
    desimal_lt = deg_to_des(lt_tempat["derajat"], lt_tempat["menit"], lt_tempat["detik"]) #derajat ke desimal
    return desimal_lt
#data Bujur tempat
def Bujur_tempat(der_bj,mnt_bj,dtk_bj):
    bj_tempat  = {"derajat": der_bj, "menit" : mnt_bj, "detik" : dtk_bj} #satuan derajat
    desimal_bj = deg_to_des(bj_tempat["derajat"], bj_tempat["menit"], bj_tempat["detik"]) #derajat ke desimal
    return desimal_bj
#Fungsi pengkodisian waktu daerah 
def nilaiC (wd) :
    if wd == "WIB":
        return 105
    elif wd == "WITA":
        return 120
    elif wd == "WIT":
        return 135
    else:
        return 0
#Fungsi jumlah hari yang diinginkan dalam jadwal waktu sholat (tanggal awal) 
def Tanggal_awal1(Th1,Bn1,Hr1):
    waktu_awal      = {"Year": Th1, "Month": Bn1, "Day": Hr1}
    tanggal_Awal2   = datetime(waktu_awal["Year"], waktu_awal["Month"], waktu_awal["Day"])
    return tanggal_Awal2
#Fungsi jumlah hari yang diinginkan dalam jadwal waktu sholat (tanggal akhir) 
def Tanggal_akhir1(Th2,Bn2,Hr2):
    waktu_akhir     = {"Year": Th2, "Month": Bn2, "Day": Hr2}
    tanggal_akhir1  = datetime(waktu_akhir["Year"], waktu_akhir["Month"], waktu_akhir["Day"])
    return tanggal_akhir1

#Implementasi Fungsi
decimal_lt    = Lintang_tempat(-8,40,00)
decimal_bj    = Bujur_tempat(115,13,10)
ketinggian    = 18 #dalam satuan meter
Wd            = nilaiC("WITA") #waktu daerah
tanggal_awal  = Tanggal_awal1(2024,12,1)
tanggal_akhir = Tanggal_akhir1(2024,12,31)
ihtiyat       = deg_to_des(0,2,0)


### DATA MATAHARI 

In [13]:
#fungsi untuk mengetahui nilai deklinasi
def deklinasi(lat, long, date, elev): 
    Bali         = ephem.Observer()
    Bali.lat     = str(lat)
    Bali.lon     = str(long)
    Bali.elev    = elev
    Bali.date    = date  
    sun          = ephem.Sun(Bali)
    deklinasi_dt = float(sun.dec) / pi * 180 
    return deklinasi_dt
#fungsi menghitung equation of time
def eot (date):
    jd  = ephem.julian_date(date)
    u   = (jd - 2451545)/36525
    l0  = 280.46607 + 36000.7698 * u
    t   = ((-1 * (1789 + 237 * u)) * m.sin(m.radians(l0)) - (7146 - 62 * u) \
        * m.cos(m.radians(l0)) + (9934 - 14 * u) * m.sin(m.radians(2 * l0)) - (29 + 5 * u) \
        * m.cos(m.radians(2 * l0)) + (74 + 10 * u) * m.sin(m.radians(3 * l0)) + (320 - 4 * u) \
        * m.cos(m.radians(3 * l0)) - 212 * m.sin(m.radians(4 * l0)))
    eot = t/1000
    return eot / 60 
#fungsi mencari nilai semi-diameter
def get_sun_semi_diameter (lat, lon, date):
    #set lokasi pengamatan
    observer      = ephem.Observer()
    observer.lat  = str(lat) #Latitude
    observer.lon  = str(lon) #Longitude
    observer.date = date 
    #Mendapatkan objek matahari
    sun           = ephem.Sun(observer)
    #Mendapatkan semi diameter
    sun_semi_diameter = sun.radius
    return sun_semi_diameter
#ini fungsi untu mendapatkan sudut waktu
def sudut_waktu (h0, dec, lt):
    t1 = m.sin(m.radians(h0))/ m.cos(m.radians(lt))/ m.cos(m.radians(dec)) - m.tan(m.radians(lt)) * m.tan(m.radians(dec))
    return m.degrees(m.acos(t1))/ 15

